In [12]:
%pip install cplex
%pip install docplex

Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.comNote: you may need to restart the kernel to use updated packages.



In [13]:
from typing import Tuple, List
import logging

def read_hgr(file_path: str) -> Tuple[int, int, List[List[int]]]:
    """
    Read a DIMACS-like .hgr file and return the number of nodes, edges, and the edge list.
    Example format:
        c I am a comment
        p hs 6 5
        1 2
        2 3 4
        2 4 5
        1 3 6
        5 6
    """
    num_nodes = 0
    num_edges = 0
    edges = []

    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()

            # Skip comments and empty lines
            if not line or line.startswith('c'):
                continue

            # Parse problem line
            if line.startswith('p'):
                parts = line.split()
                if len(parts) != 4 or parts[1] != 'hs':
                    raise ValueError(f"Invalid problem line format: {line}")
                num_nodes = int(parts[2])
                num_edges = int(parts[3])
                continue

            # Parse edge line
            vertices = [int(v) for v in line.split()]
            if not vertices:
                continue

            # Validate vertex numbers
            if any(v < 1 or v > num_nodes for v in vertices):
                raise ValueError(f"Vertex number out of range in line: {line}")

            edges.append(vertices)

    if not num_nodes or not num_edges:
        raise ValueError("Problem line not found or invalid")

    if len(edges) != num_edges:
        logging.warning(f"Expected {num_edges} edges but found {len(edges)}")

    return num_nodes, num_edges, edges


import docplex.mp.model as cplex
import json

def solve_ilp(num_nodes, num_edges, edges, param_dict={}, print_info=False):
    mdl = cplex.Model()

    picked_vertices = mdl.binary_var_list(num_nodes, name="picked_vertices")

    for idx, edge in enumerate(edges):
        mdl.add_constraint(mdl.sum(picked_vertices[node-1] for node in edge) >= 1, "Edge_" + str(idx))

    mdl.minimize(mdl.sum(picked_vertices))

    """Set parameters on the model using a dictionary."""
    for param_path, value in param_dict.items():
        parts = param_path.split('.')
        param = mdl.parameters
        for part in parts:
            param = getattr(param, part)
        param.set(value)

    solution = mdl.solve()
    if print_info:
        mdl.print_information()
        print(mdl.parameters._params)
        print(mdl.solve_details)
        print(f"Solution size: {solution.objective_value}")
        print(f"Sol: {solution}")
        # print(vars(mdl.solve_details))
        # print(dir(mdl.solve_details))
    result = vars(mdl.solve_details)
    result['config'] = param_dict
    result['objective'] = solution.objective_value if solution else None

    return result

In [14]:
FOLDER = 20
import os
if not os.path.exists(f"{FOLDER}"):
	os.mkdir(f"{FOLDER}")
num_nodes, num_edges, edges = read_hgr(f"./testset/bremen_subgraph_{FOLDER}.hgr")
print(f"Graph has {num_nodes} nodes and {num_edges} edges")
solve_ilp(num_nodes, num_edges, edges, print_info=True)

Graph has 32 nodes and 31 edges
Model: docplex_model723
 - number of variables: 32
   - binary=32, integer=0, continuous=0
 - number of constraints: 31
   - linear=31
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
[docplex.mp.params.IntParameter(parameters.advance,1), docplex.mp.params.IntParameter(parameters.clocktype,2), docplex.mp.params.NumParameter(parameters.dettimelimit,1e+75), docplex.mp.params.IntParameter(parameters.lpmethod,0), docplex.mp.params.IntParameter(parameters.optimalitytarget,0), docplex.mp.params.IntParameter(parameters.parallel,0), docplex.mp.params.BoolParameter(parameters.paramdisplay,1), docplex.mp.params.IntParameter(parameters.qpmethod,0), docplex.mp.params.IntParameter(parameters.randomseed,202009243), docplex.mp.params.BoolParameter(parameters.record,0), docplex.mp.params.IntParameter(parameters.solutiontype,0), docplex.mp.params.IntParameter(parameters.threads,0), docplex.mp.params.NumParameter(parameters.timelimit,1e+75), docplex

{'_time': 0.0319999999992433,
 '_dettime': 0.579620361328125,
 '_solve_status_code': 101,
 '_solve_status': 'integer optimal solution',
 '_problem_type': 'MILP',
 '_ncolumns': 32,
 '_linear_nonzeros': 125,
 '_miprelgap': 0.0,
 '_best_bound': 9.0,
 '_n_iterations': 118,
 '_n_nodes_processed': 0,
 '_quality_metrics': {},
 'config': {},
 'objective': 9.0}

In [15]:
from tqdm.contrib.itertools import product
import tqdm.notebook as tqdm
# [docplex.mp.params.IntParameter(parameters.advance,1), docplex.mp.params.IntParameter(parameters.clocktype,2), docplex.mp.params.NumParameter(parameters.dettimelimit,1e+75), docplex.mp.params.IntParameter(parameters.lpmethod,0), docplex.mp.params.IntParameter(parameters.optimalitytarget,0), docplex.mp.params.IntParameter(parameters.parallel,0), docplex.mp.params.BoolParameter(parameters.paramdisplay,1), docplex.mp.params.IntParameter(parameters.qpmethod,0), docplex.mp.params.IntParameter(parameters.randomseed,202009243), docplex.mp.params.BoolParameter(parameters.record,0), docplex.mp.params.IntParameter(parameters.solutiontype,0), docplex.mp.params.IntParameter(parameters.threads,0), docplex.mp.params.NumParameter(parameters.timelimit,1e+75), docplex.mp.params.StrParameter(parameters.workdir,.), docplex.mp.params.NumParameter(parameters.workmem,2048.0)]

import pandas as pd
import numpy as np

NR_RUNS = 30
def gridsearch(param_values, runs_per_data = NR_RUNS):
	rezults = []
	param_names = list(param_values.keys())
	for cfg in product(*param_values.values()):
		# Remove None values (-> defaults)
		cfg = {param_names[i]: v for i, v in enumerate(cfg) if v is not None}
		runs = [solve_ilp(num_nodes, num_edges, edges, param_dict=cfg) for _ in tqdm.trange(runs_per_data, position=2, leave=False)]
		rezults += runs
	return rezults

In [ ]:
param_values_bnb = {
	"threads": [1],
	"mip.strategy.search": [1],  # b&b
	# "lpmethod": [1, 2],  # primal, dual
    "mip.cuts.cliques": [-1],  # auto, moderate
    "mip.cuts.gomory": [-1],  # off, moderate, aggressive
    "mip.cuts.mircut": [-1],  # off, moderate, aggressive
    "mip.cuts.liftproj": [-1],  # off, moderate, aggressive
    "mip.cuts.zerohalfcut": [-1],  # off, moderate, aggressive
    "mip.cuts.covers": [-1, 1, 2],  # off, moderate, aggressive, very aggressive
    # "emphasis.mip": [0, 1, 2],  # balanced, feasibility, optimality
    # "mip.strategy.probe": [0, 1, 2, 3],  # auto, limited, moderate, full
    # "mip.strategy.heuristiceffort": [0.5, 1.0, 2.0],  # low, default, high
    "mip.strategy.variableselect": [-1, 1, 2, 3],  # min inf, auto, max inf, pseudo-cost, strong
}
for folder in [200]:#[20, 50, 100, 150, 200, 250, 300]:
	num_nodes, num_edges, edges = read_hgr(f"./testset/bremen_subgraph_{folder}.hgr")
	rezults_bnb = gridsearch(param_values_bnb)
	rezults_bnb = pd.DataFrame(rezults_bnb)
	display(rezults_bnb)
	rezults_bnb.to_json(f"{folder}/rezults_bnbt1.json", indent=2)

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]